# Purpose
This notebook tests the naive training of the the completion network.

# Import

In [1]:
import torch
import torchvision

from glic.networks.completion_network import CompletionNetwork
from glic.training.cn_training import train_cn
from glic.utils import *

# Train

In [2]:
# load the dataset
train_dataset_dir = '../data/train/'
train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_dir, transform=torchvision.transforms.ToTensor())
train_dataloader = get_dataloader(train_dataset_dir, train_dataset_dir+"amphitheater/00000005.jpg", batch_size=4)

In [3]:
# others
cn = CompletionNetwork().cuda()
optimizer = torch.optim.Adadelta(cn.parameters(), lr=2e-4)
num_batch = 10
replacement_val = torch.rand(3)

In [4]:
# train
loss_list = train_cn(cn, optimizer, train_dataloader, num_batch, replacement_val)


### BATCH 0 ###
loss: 3572.98583984375

### BATCH 1 ###


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 2.00 GiB total capacity; 1.66 GiB already allocated; 0 bytes free; 1.71 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# save checkpoint
resume_path = update_resume_path(train_dataset_dir, "../data/train/airfield/00000001.jpg", num_batch, 4)
save_checkpoint("../logs/checkpoints/",
    cn,
    optimizer,
    [loss_list],
    num_batch,
    resume_path,
    replacement_val=replacement_val)